In [1]:
"""A module that makes downloading and using many GSE datasets fast and easy.

Author: Joshua Blanchard, last update: 4/28/2021

A typical workflow:

    Download wanted GSE family files using the download() function.

    Extract content from these files using the extract() function.

    Use the family_dict() function to convert data to pandas.DataFrame objects.

    With the info() function, perform data analysis on the previously created pandas.DataFrame objects.
"""

'A module that makes downloading and using many GSE datasets fast and easy.\n\nAuthor: Joshua Blanchard, last update: 4/28/2021\n\nA typical workflow:\n\n    Download wanted GSE family files using the download() function.\n\n    Extract content from these files using the extract() function.\n\n    Use the family_dict() function to convert data to pandas.DataFrame objects.\n\n    With the info() function, perform data analysis on the previously created pandas.DataFrame objects.\n'

In [2]:
import pandas as pd
import os
import numpy as np
import re as re
import xmltodict
import shutil
import ftplib
import tarfile
import pickle
import gzip
# import requests
# import wget

In [3]:
__base_path = "data"

In [4]:
__hidden_path = ".aidp_files"

In [5]:
def family_dict(GSE_family):
    
    """
    Given a family ID, will output a dictionary. Keys will be the sample IDs, values will be the corresponding
    pandas.DataFrame object.
    """

#     let's check if we already have this dictionary saved
    if not os.path.exists("./" + __hidden_path):
        os.mkdir(__hidden_path)
    
    dict_path = __hidden_path + "/" + GSE_family + "_dict"    
    if not os.path.exists(dict_path):
    
        family_directory = __family_path(GSE_family)
        total_list = os.listdir(family_directory)
        valid_files = []

        for file_name in total_list:
            match = re.match(r"GSM", file_name)
            if match:
                valid_files.append(file_name)

        family_dict = {}
        for file_name in valid_files:
            file_df = __load_file(os.path.join(family_directory, file_name))
            sample_id = re.match(r"GSM\d+", file_name).group(0)
            family_dict[sample_id] = file_df
            
        dict_file = open(dict_path, 'wb')
        pickle.dump(family_dict, dict_file)
        dict_file.close()
        
    else:
        dict_file = open(dict_path, 'rb')
        family_dict = pickle.load(dict_file)
        dict_file.close()
        
        
    return family_dict


In [6]:
def __family_path(GSE_family):
    
    """
    Exists to build paths to the family's directory.
    
    I downloaded the files using the download() function in conjuction with the extract() function.
    """
        
    return os.path.join(__base_path, GSE_family + "/")


In [7]:
def __load_file(file_directory):
#     clean data file
#     convert to a dataframe object and return

    """
    Given a file name will output a corresponding pandas.DataFrame object.
    """
    
    try:
        clean_dict = __clean(file_directory)
    except PermissionError:
        print("You likely inputted the path of a directory, not a file.")
    
#     return pd.DataFrame({"site": clean_dict["col_1"], "measurement": clean_dict["col_2"]})
    return pd.DataFrame(data= clean_dict["col_2"], index= clean_dict["col_1"], columns= ["measurement"])


In [8]:
def __clean(file_path):
    
    """
    Cleans a given .txt file.
    
    Returns a dictionary:
    
    "site": first column
    "measurement": second column
    "bad_rows": list of all the invalid rows
    """

    valid_rows = []
    not_valid_rows = []
    file = open(file_path, 'r')
    
    for line in file:
        
#         checks for only the first two columns
        line_match = re.match(r"\S+\t\S+", line)
        if line_match:
            valid_rows.append(line_match.group(0))
        else:
            not_valid_rows.append(line)
        
    file.close()
    
#     now let's split our valid_rows list into two lists, one for each column
    col_1 = []
    col_2 = []
    for row in valid_rows:
        row_match = re.match(r"(\S+)\t(\S+)", row)
        col_1.append(row_match.group(1))
        col_2.append(row_match.group(2))
        
    return {"col_1":col_1, "col_2":col_2, "bad_rows":not_valid_rows}


In [9]:
def info(GSE_family, sample_id, info):
    
    """
    Given a GSE family and the ID of the wanted sample will return the desired information of the sample.
    
    Possible values for the info parameter:
    
    "age": the unit of the outputted age will always be years.
    """
#     create dataframe of sample characteristics portion of the series_matrix file
# read desired info from this dataframe. will have to use regex to find the right information
# set up some sort of persistance of this dataframe for faster retrieval of information

#             match = re.search(r"(a|A)(g|G)(e|E)", tag)

    


In [ ]:
def info(sample_id, info):
    
    """
    Given a GSE family and the ID of the wanted sample will return the desired information of the sample.
    
    Possible values for the info parameter:
    
    "age": the unit of the outputted age will always be years.
    """
#     create dataframe of sample characteristics portion of the series_matrix file
# read desired info from this dataframe. will have to use regex to find the right information
# set up some sort of persistance of this dataframe for faster retrieval of information

#             match = re.search(r"(a|A)(g|G)(e|E)", tag)

In [30]:
test_df = __matrix_to_df("./testing-data/GSE17448_series_matrix.txt", use= "info()")
test_series = test_df.iloc[:, 0]

for row in test_series:
    print(row)

cell type: mesenchymal stromal cells from human bone marrow
age: 50 year old human donor
cell passage: 2


In [10]:
for row in test_series:
    match_age = re.search(r"(a|A)(g|G)(e|E)", row)
    if match_age:
#         let's grab the age
        match_age_dig = re.search(r"\d+\.?\d*", row)
        age_float = float(match_age_dig.group())
        
#         now let's find the unit of age
        match_years = re.search(r"(y|Y)(e|E)(a|A)(r|R)(s|S)*", row)
        match_months = re.search(r"(m|M)(o|O)(n|N)(t|T)(h|H)(s|S)*", row)
        match_days = re.search(r"(d|D)(a|A)(y|Y)(s|S)*", row)
        match_hours = re.search(r"(h|H)(o|O)(u|U)(r|R)(s|S)*", row)
        
#         we always return the age in units of years
        if match_years:
            return age_float
        elif match_months:
            return age_float / 12
        elif match_days:
            return age_float / 365
        elif match_hours:
            return age_float / 8760
        else:
#             if no unit is given we'll default to years
            return age_float


In [36]:
float("50") / 10

5.0

In [27]:
series("GSE27317").head(3)

,GSM675231,GSM675232,GSM675233,GSM675234,GSM675235,GSM675236,GSM675237,GSM675238,GSM675239,GSM675240,...,GSM675389,GSM675390,GSM675391,GSM675392,GSM675393,GSM675394,GSM675395,GSM675396,GSM675397,GSM675398
ID_REF,,,,,,,,,,,,,,,,,,,,,
cg00000292,0.8850,0.8630,0.8150,0.872,0.8900,0.8510,0.9060,0.837,0.8720,0.7700,...,0.836,0.7930,0.8580,0.747,0.7750,0.8130,0.8070,0.8390,0.8320,0.769
cg00002426,0.7900,0.7730,0.6830,0.826,0.7940,0.7400,0.7310,0.737,0.7380,0.6980,...,0.905,0.8960,0.8980,0.895,0.8650,0.8960,0.9050,0.9170,0.8940,0.885
cg00003994,0.0734,0.0651,0.0577,0.074,0.0805,0.0648,0.0568,0.119,0.0533,0.0539,...,0.073,0.0497,0.0434,0.032,0.0371,0.0306,0.0245,0.0291,0.0346,0.032


In [25]:
def series(GSE):
    
    download(GSE, file_type= "series_matrix")
    extract()
    
    file_path = os.path.join(__base_path, GSE, GSE + "_series_matrix.txt")
    
    return __matrix_to_df(file_path)
    
    
# download and extract .soft file using the download() and extract() function
# convert file to a dataframe

In [13]:
# help(__matrix_to_df)

In [14]:
def __matrix_to_df(file_path, use= "series()"):
    
    """Returns the pandas.dataframe corresponding to the series_matrix file.
    
    Possible values for use: "series()", "info()"
    """
    
    start_row, num_rows = __matrix_helper(file_path, use)
    df = pd.read_csv(file_path, header= start_row, sep= "\t", low_memory= False, nrows= num_rows)
    
    if use == "series()":
        df.set_index("ID_REF", inplace= True)
    else:
        df.set_index("!Sample_geo_accession", inplace= True)
        df = df.loc["!Sample_characteristics_ch1"]
    
    return df

In [15]:
def __matrix_helper(file_path, use= "series()"):
    
    """Returns a tuple containing the start line for reading (0) and the number of rows to read (1).
    
    Possible values for use: "series()", "info()"
    """
    
    file = open(file_path)
    
    line_num = 0
    if use == "series()":
        
        while True:
            line = file.readline()

            if line == "!series_matrix_table_begin\n":
                start_row = line_num
            elif line == "!series_matrix_table_end\n":
                end_row = line_num - 1
            elif line == "":
                break

            line_num += 1
        
    else:
        
        while True:
            line = file.readline()

            if "!Sample_title" in line:
                start_row = line_num
            elif line == "!series_matrix_table_begin\n":
                end_row = line_num - 1
            elif line == "":
                break

            line_num += 1

    num_rows = end_row - start_row - 1
        
    return start_row, num_rows

In [16]:
def __xml_path(GSE_family):
    
    """
    Exists to build a path to the family's .xml.
    
    I downloaded these families using the download() function in conjuction with the extract() function.
    """
    
    return os.path.join(__base_path, GSE_family + "/" + GSE_family + "_family.xml")
    

In [17]:
def __xml_to_dict(xml_path):
    
    """
    Exists to convert a .xml file at xml_path to a dictionary.
    """
    
    family_file = open(xml_path,'r+b')
    family_dict = xmltodict.parse(family_file)
    family_file.close()
    
    return family_dict


In [18]:
def __dict_index(GSE_family, sample_id):
    
    """
    Gives the index of where in the dictionary the sample's information is.
    """
    
    return __sample_indices(GSE_family)[sample_id]


In [19]:
def __sample_indices(GSE_family):
    
    """
    Exists to return the indices of each sample's information within the associated family dictionary. Returns a dictionary
    with keys equal to the sample ID ("GSM***") and values equal to the index of that sample's information within the family
    dictionary.
    """
    
    family_dir = __family_path(GSE_family)
    file_list = os.listdir(family_dir)
    
    filtered_list = []
    for file_name in file_list:
        sample_match = re.match(r"GSM", file_name)
        if sample_match:
            filtered_list.append(file_name)
            
    for i in np.arange(len(filtered_list)):
        filtered_list[i] = re.match(r"GSM\d+", filtered_list[i]).group(0)
        
    index_dict = {}
    index = 0

    for sample_id in filtered_list:
        index_dict[sample_id] = index
        index += 1
        
    return index_dict


In [20]:
# url = "ftp://ftp.ncbi.nlm.nih.gov/geo/series/GSEnnn/" + "GSE41037" + "/miniml/" + "GSE41027" + "_family.xml.tgz"
# url = "ftp://ftp.ncbi.nlm.nih.gov/geo/series/GSEnnn/GSE1/miniml/GSE1_family.xml.tgz"

def download(GSE_family_list, file_type= "miniml"):
    
    """
    Will download family .tgz files to the following directory: "./data/"
    
    Possible values for file_type: "miniml", "series_matrix"
    """
    
    if (type(GSE_family_list) == str):
        GSE_family_list = [GSE_family_list]

    url = "ftp.ncbi.nlm.nih.gov"
    ftp = ftplib.FTP(url)
    ftp.login()
    
    if not os.path.exists(__base_path):
        os.mkdir(__base_path)
    
    for GSE_family in GSE_family_list:
        
        if file_type == "miniml":
        
            ftp.cwd("/geo/series/" + __sub_directory(GSE_family) + "/" + GSE_family + "/miniml/")
            filename = GSE_family + "_family.xml.tgz"
            
            if not (os.path.exists(__base_path + "/" + filename) or os.path.exists(__base_path + "/" + GSE_family)):
                local_file = open(__base_path + "/" + filename, 'wb')
                ftp.retrbinary('RETR ' + filename, local_file.write, blocksize= 16_384)
                local_file.close()
            
        else:
            
            ftp.cwd("/geo/series/" + __sub_directory(GSE_family) + "/" + GSE_family + "/matrix/")
            filename = GSE_family + "_series_matrix.txt.gz"
        
            if not (os.path.exists(__base_path + "/" + filename) or os.path.exists(__base_path + "/" + GSE_family + "/" + GSE_family + "_series_matrix.txt")):
                local_file = open(__base_path + "/" + filename, 'wb')
                ftp.retrbinary('RETR ' + filename, local_file.write, blocksize= 16_384)
                local_file.close()

    ftp.quit()
    

In [21]:
def __sub_directory(GSE_ID):
    
    """
    Given a GSE ID will return the corresponding sub-directory.
    """
    
    gse_int = __ID_to_int(GSE_ID)
    
    if gse_int <= 171:
        ret_str = "GSE" + str(gse_int) + "nnn"
    else:
        first_3_dig = int(str(gse_int)[0:3])
        if first_3_dig <= 171:
            ret_str = "GSE" + str(first_3_dig) + "nnn"
        else:
            first_2_dig_str = str(gse_int)[0:2]
            ret_str = "GSE" + first_2_dig_str + "nnn"
            
    return ret_str


In [22]:
def __ID_to_int(GSE_ID):
    
    """
    Given some GSE ID will return the corresponding integer as an int.
    
    Example:
    
    __ID_to_int("GSE41037") will return 41037.
    
    """
    
    match = re.search(r"\d+", GSE_ID)
    
    if not match:
        print(GSE_ID)
    
    return int(match.group(0))


In [23]:
def extract():
    
    """
    Will extract files from all downloaded family .tgz files to a respective directory: ./data/GSE***/
    
    This will also delete the compressed .tgz files.
    """
    
#     find all .tgz and .gz files
# extract those files
# delete the .tgz files

    file_list = os.listdir(__base_path)
    tgz_list = []
    gz_list = []
    
    for filename in file_list:
        match_tgz = re.search(r"\.tgz", filename)
        match_gz = re.search(r"\.gz", filename)
        
        if match_tgz:
            tgz_list.append(filename)
        elif match_gz:
            gz_list.append(filename)
                
    for filename in tgz_list:
        full_path = __base_path + "/" + filename
        family_id = re.search(r"GSE\d+", filename).group(0)
                
        file = tarfile.open(full_path)
        
        flag = False
        try:
            out_path = "./" + __base_path + "/" + family_id
            file.extractall(out_path)
        except:
#             let's end the content extraction of the file. will fix later.
            file.close()
    
#             out_file = open(out_path)
#             out_file.close()
            
#             os.remove(out_path)
            
            flag = True
            print("Something weird happened while extracting from the " + family_id + " compressed file. Ended extraction early for " + family_id + ".")
            
        if not flag:
            file.close()
            os.remove(full_path)
            
    
    for filename in gz_list:
        full_path = "./" + __base_path + "/" + filename
        family_id = re.search(r"GSE\d+", filename).group(0)
                
        compressed_file = open(full_path, 'rb')
        compressed_file_contents = compressed_file.read()
        compressed_file.close()
        
        contents_bytes = gzip.decompress(compressed_file_contents)
        contents_str = contents_bytes.decode()
        
        family_dir = os.path.join(__base_path, family_id)
        if not os.path.exists(family_dir):
            os.mkdir(family_dir)
        
        out_path = "./" + __base_path + "/" + family_id + "/" + family_id + "_series_matrix.txt"

        file = open(out_path, 'w')
        file.write(contents_str)
        file.close()
        
        os.remove(full_path)